In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import cv2
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
path = '../input/bengaliai-cv19/train_image_data_0.parquet'
data = pd.read_parquet(path)

In [ ]:
data.head()

In [ ]:
label_path = '../input/bengaliai-cv19/train.csv'

In [ ]:
label_df = pd.read_csv(label_path)

In [ ]:
grapheme_1 = label_df.groupby('grapheme').get_group('প্র')
grapheme_1.head()

In [ ]:
df_lexicon = pd.read_csv('../input/indicword/indicword_bengali_lexicon_grapheme.csv')
df_lexicon.head()

In [ ]:
def get_data(df,ind):
    img_id = df.iloc[ind].values[0]
    img = df.iloc[ind].values[1:].reshape(137,236).astype(np.uint8)
    return img, img_id

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def get_img_bb(img):
    kernel = np.ones((5,5),np.uint8)
    opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    mask = opening<(255//2)

    maskx = np.any(mask, axis=0)
    masky = np.any(mask, axis=1)
    x1 = np.argmax(maskx)
    y1 = np.argmax(masky)
    x2 = len(maskx) - np.argmax(maskx[::-1])
    y2 = len(masky) - np.argmax(masky[::-1])
    sub_image = img[y1:y2, x1:x2]
    return sub_image, (y1,y2,x1,x2)

In [ ]:
img_bb = []
ind = 10
print(df_lexicon.iloc[ind]['lexicon'])
graphemes = df_lexicon.iloc[ind]['graphemes'].split('-')
indices = [label_df.groupby('grapheme').get_group(g).index[2] for g in graphemes]
for ind in indices:
    img, img_id = get_data(data,ind)
#     print(img_id)
    sub_image,_ = get_img_bb(img)
    img_bb.append(sub_image)
    
img_start = img_bb[0]
for img in img_bb[1:]:
    if img.shape[0]>img_start.shape[0]:
        pad = img.shape[0]-img_start.shape[0]
        img_start = np.concatenate(
            (img_start,np.ones((pad,img_start.shape[1]),dtype=np.uint8)*255),
            axis = 0
        )
        
    elif img.shape[0]<img_start.shape[0]:
        pad = img_start.shape[0] - img.shape[0]
        img = np.concatenate(
            (img,np.ones((pad,img.shape[1]),dtype=np.uint8)*255),
            axis = 0
        )
    img_start = np.concatenate((img_start,img), axis = 1)
plt.imshow(img_start)
plt.show()